# 해당 파일 설명
이 코드를 실행하기 위한 파일, /convert 경로에 다음의 파일이 필요합니다.

1. ALL_FC_DATA_MEAN.csv
2. data_all_time_out_in.csv

이 코드를 모두 실행하고 나면, /convert 경로에 다음의 파일이 생성됩니다.

1. ALL_FC_DATA_MEAN_ADD_WEATHER.csv

이 코드를 모두 실행하고 나면, /model 경로에 다음의 파일이 생성됩니다.

1. model_fc_all.pkl

## 추가 설명
예보 데이터를 기반으로 weather 데이터를 예측합니다. 

이때 굉장히 높은 정확도를 갖는 모델을 만들었습니다.

그 모델을 model_fc_all.pkl 로 저장합니다.

# 1. Google Drive Mount

In [ ]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# 2. Define Directory

In [ ]:
# 사용하기 전 변경해주어야 하는 파라미터
BASE_WORKING_DIR = "/content/gdrive/My Drive/weather_bigcontest/4_code/"
USERNAME = "park"

In [ ]:
import os
# 사용하는 디렉토리 정의
DATA_BASE_PATH = os.path.join(BASE_WORKING_DIR, "data")
DATA_RAW_PATH = os.path.join(DATA_BASE_PATH, "raw")
DATA_EXPORT_PATH = os.path.join(DATA_BASE_PATH, "export")
DATA_CONV_USER_PATH = os.path.join(DATA_BASE_PATH, USERNAME)
DATA_CONV_PATH = os.path.join(DATA_BASE_PATH, "converted")
MODEL_PATH = os.path.join(BASE_WORKING_DIR, "model")
RESULT_PATH = os.path.join(BASE_WORKING_DIR, "result")

WORKING_DIR = os.path.join(BASE_WORKING_DIR, USERNAME)

print('Current Work Directory :',  os.getcwd(), '\n')
print('Username :',  USERNAME, '\n')

print("DATA_BASE_PATH : ", DATA_BASE_PATH)
print("DATA_RAW_PATH : ", DATA_RAW_PATH)
print("DATA_EXPORT_PATH : ", DATA_EXPORT_PATH)
print("DATA_CONV_USER_PATH : ", DATA_CONV_USER_PATH)
print("DATA_CONV_PATH : ", DATA_CONV_PATH)

print("MODEL_PATH : ", MODEL_PATH)

print("RESULT_PATH : ", RESULT_PATH)

print("WORKING_DIR : ", WORKING_DIR)

Current Work Directory : /content 

Username : park 

DATA_BASE_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data
DATA_RAW_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/raw
DATA_EXPORT_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/export
DATA_CONV_USER_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/park
DATA_CONV_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/data/converted
MODEL_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/model
RESULT_PATH :  /content/gdrive/My Drive/weather_bigcontest/4_code/result
WORKING_DIR :  /content/gdrive/My Drive/weather_bigcontest/4_code/park


In [ ]:
# 데이터 무엇이 있는지 확인하자.
# 아래 코드를 수행하게 되면, "../input/"에서 리눅스 명령어 "ls"를 수행한 결과를 보여주게 된다. 이 때 나온 binary code를 "utf8"로 디코딩해서 보여준다.
from subprocess import check_output
print("===== Raw Data List =====\n", check_output(["ls", DATA_RAW_PATH]).decode("utf8"))
print("===== Export Data List =====\n" ,check_output(["ls", DATA_EXPORT_PATH]).decode("utf8"))
print("===== Converted Data List =====\n" ,check_output(["ls", DATA_CONV_PATH]).decode("utf8"))

===== Raw Data List =====
 plant1_train.csv
plant2_train.csv
plant2_train.gsheet
plant_test.csv
submit.csv
submit.gsheet

===== Export Data List =====
 dangjin_aws
forecast_data
seosan_asos
sinpyeong_aws
songak_forecast
songak_local
songsan_forecast
songsan_local

===== Converted Data List =====
 ALL_FC_DATA_MEAN_ADD_WEATHER.csv
ALL_FC_DATA_MEAN.csv
auc_95_temp_save
backup
dangjin_weather.csv
data_all_time_out.csv
data_all_time_out_in_add_mea_ddhr.csv
data_all_time_out_in.csv
data_all_time_predict.csv
data_plant_train_trend_point.csv
date_trend_point.csv
FC_data
old_local
plant1_by_loc_with_datetime.csv
plant2_by_loc_with_datetime.csv
plant_test_all_weather_feature_add_cluster_add_out.csv
plant_test_all_weather_feature_add_cluster_add_out_in.csv
plant_test_all_weather_feature_add_cluster.csv
plant_test_all_weather_feature.csv
plant_test_with_datetime.csv
plant_train_all_weather_feature_add_cluster.csv
plant_train_all_weather_feature.csv
seosan_weather.csv
sinpyeong_weather.csv
songak_w

# 4. Import Library

In [ ]:
# 기본 데이터 라이브러리 로드
import sys # 시스템 파라미터에 접근할 수 있게 도와준다.
print("Python version: {}". format(sys.version))

import pandas as pd # 데이터 정제에 도움을 주는 라이브러리
print("pandas version: {}". format(pd.__version__))

import matplotlib # 매트랩에서 사용하는 시각화 도구를 사용할 수 있게 도와주는 시각화 도구
print("matplotlib version: {}". format(matplotlib.__version__))

import numpy as np # 행렬 계산을 위해 필요한 라이브러리
print("NumPy version: {}". format(np.__version__))

import scipy as sp # 수학 관련 함수가 내장된 라이브러리
print("SciPy version: {}". format(sp.__version__)) 

import IPython
from IPython import display # 주피터 노트북에서 예쁘게 시각화 해주는 도구
print("IPython version: {}". format(IPython.__version__)) 

import sklearn # 각종 통계 도구와 머신 러닝 알고리즘이 내장되어 있는 라이브러리
print("scikit-learn version: {}". format(sklearn.__version__))

# 파이썬 내장 라이브러리
import random
import time
import os


# Jupyter Notebook 이나 ipython 을 사용하다보면 향후 버전이 올라갈 때 변경될 사항 등을 알려주는 경고 메시지(warning message)를 뜨지 않게 해준다.
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

Python version: 3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]
pandas version: 1.0.5
matplotlib version: 3.2.2
NumPy version: 1.18.5
SciPy version: 1.4.1
IPython version: 5.5.0
scikit-learn version: 0.22.2.post1
-------------------------


In [ ]:
# 모델링 라이브러리, 수학 계산 툴 로드
# 일반적인 모델링 라이브러리
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler  # 표준화 패키지 라이브러리 

# 모델링 시 헬퍼 함수들
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans

# interpolation 함수
import scipy.interpolate as spi

# model load
import pickle
from sklearn.externals import joblib

# 시각화 도구
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.plotting import scatter_matrix

# 시각화 도구 default 세팅
# 주피터 노트북에서 plot 결과를 볼 수 있게 해준다.
%matplotlib inline
mpl.style.use('ggplot') # matplotlib에서 plot되는 결과를 선택할 수 있다.
sns.set_style('white') # seaborn에서 사용할 style을 설정할 수 있다.
pylab.rcParams['figure.figsize'] = 12,8 # plot의 크기와 선 등의 기본 값을 설정할 수 있다.

# 시각화 Style 설정하기
# print(plt.style.available)
# ['bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'seaborn-bright', 'seaborn-colorblind', 'seaborn-dark-palette', 'seaborn-dark', 'seaborn-darkgrid', 'seaborn-deep', 'seaborn-muted', 'seaborn-notebook', 'seaborn-paper', 'seaborn-pastel', 'seaborn-poster', 'seaborn-talk', 'seaborn-ticks', 'seaborn-white', 'seaborn-whitegrid', 'seaborn', 'Solarize_Light2', 'tableau-colorblind10', '_classic_test']

plt.style.use(['fivethirtyeight'])
sns.set_style('darkgrid')

# 5. Read Raw Data

In [ ]:
FC_DATA = pd.read_csv(os.path.join(DATA_CONV_PATH, 'ALL_FC_DATA_MEAN.csv'))
FC_DATA.drop(columns=['Unnamed: 0'], inplace=True)
FC_DATA['time'] = FC_DATA['time'].apply(lambda x : int(x))
FC_DATA

,date,time,songsan_ta,songsan_hm,songsan_ws,songak_ta,songak_hm,songak_ws,sinpyeong_ta,sinpyeong_hm,sinpyeong_ws,dangjin1_ta,dangjin1_hm,dangjin1_ws,dangjin2_ta,dangjin2_hm,dangjin2_ws,dangjin3_ta,dangjin3_hm,dangjin3_ws,godae_ta,godae_hm,godae_ws,emam_ta,emam_hm,emam_ws,susuk_ta,susuk_hm,susuk_ws,suknam_ta,suknam_hm,suknam_ws
0,2016-03-01,900,1.900000,43.000000,2.9,2.000000,43.000000,2.8,1.900000,42.0,2.5,2.0,45.0,3.0,2.0,45.0,3.0,2.0,45.0,3.0,1.800000,46.0,3.0,2.0,45.0,3.0,2.200000,46.0,3.0,2.200000,46.0,3.0
1,2016-03-01,1080,0.200000,63.000000,3.9,0.100000,63.000000,3.8,0.500000,60.0,3.4,0.0,65.0,4.0,0.0,65.0,4.0,0.0,65.0,4.0,0.100000,65.0,4.0,0.0,65.0,4.0,0.300000,64.0,3.9,0.300000,64.0,3.9
2,2016-03-01,1260,-0.933333,78.666667,1.9,-1.133333,78.666667,1.8,-1.166667,77.0,1.5,-1.0,80.0,2.0,-1.0,80.0,2.0,-1.0,80.0,2.0,-0.766667,79.0,1.9,-1.0,80.0,2.0,-0.433333,79.0,2.0,-0.433333,79.0,2.0
3,2016-03-02,0,-1.050000,79.500000,1.0,-1.200000,80.000000,1.0,-1.225000,79.5,1.0,-1.0,80.0,1.0,-1.0,80.0,1.0,-1.0,80.0,1.0,-0.850000,78.5,1.0,-1.0,80.0,2.0,-0.450000,79.0,2.0,-0.450000,79.0,2.0
4,2016-03-02,180,-1.300000,80.400000,1.9,-1.580000,80.400000,1.9,-1.920000,79.2,1.6,-1.2,80.0,2.0,-1.2,80.0,2.0,-1.2,80.0,2.0,-0.940000,78.6,1.9,-1.2,80.0,2.0,-0.600000,78.0,2.0,-0.600000,78.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11951,2020-04-03,720,14.000000,35.000000,3.0,14.000000,35.000000,2.8,14.000000,35.0,2.8,14.0,35.0,2.3,14.0,35.0,2.3,14.0,35.0,2.3,14.000000,35.0,2.8,14.0,30.0,2.9,13.000000,40.0,2.9,13.000000,40.0,2.9
11952,2020-04-03,900,16.000000,25.000000,4.5,15.000000,25.000000,4.2,15.000000,25.0,4.3,15.0,30.0,3.1,15.0,30.0,3.1,15.0,30.0,3.1,15.000000,30.0,4.3,15.0,30.0,4.2,13.000000,35.0,4.2,13.000000,35.0,4.2
11953,2020-04-03,1080,12.000000,40.000000,4.1,12.000000,40.000000,3.7,12.000000,40.0,3.7,11.0,40.0,2.3,11.0,40.0,2.3,11.0,40.0,2.3,11.000000,45.0,3.9,11.0,45.0,3.4,10.000000,50.0,3.7,10.000000,50.0,3.7
11954,2020-04-03,1260,8.000000,70.000000,2.6,8.000000,65.000000,2.6,8.000000,65.0,2.3,8.0,70.0,1.5,8.0,70.0,1.5,8.0,70.0,1.5,8.000000,75.0,2.8,7.0,75.0,1.7,7.000000,80.0,2.8,7.000000,80.0,2.8


In [ ]:
data = pd.read_csv(os.path.join(DATA_CONV_PATH, 'data_all_time_out_in.csv'))
data.drop(columns=['Unnamed: 0'], inplace=True)
data['time'] = data['time'].apply(lambda x : int(x))
data

,date,plant,loc,time,dangjin-ta,dangjin-hm,dangjin-ws,dangjin-rn,sinpyeong-ta,sinpyeong-ws,sinpyeong-rn,sinpyeong-lp,sinpyeong-ap,seosan-ta,seosan-hm,seosan-ws,seosan-rn,seosan-dew,seosan-sp,seosan-lp,seosan-ap,seosan-sr,seosan-ss,seosan-ground,songsan-ta,songsan-hm,songsan-ws,songsan-rn,songak-ta,songak-hm,songak-ws,songak-rn,seosan-ta-trend,seosan-hm-trend,seosan-ground-trend,seosan-ta-point,seosan-hm-point,seosan-ground-point,seosan-ta-changed,seosan-hm-changed,seosan-ground-changed,tem_out_loc,hum_out_loc,tem_in,hum_in,tem_coil
0,2016-04-01,1,1,0,6.8,41.2,0.3,0.0,7.2,0.9,0.0,1013.0,1015.7,8.8,41.0,1.5,0.0,-3.7,4.6,1012.1,1015.6,0.0,0.00,4.0,14.000000,40.0,0.6,0.0,13.800000,38.0,0.4,0.0,4,4,3,2,1,0,18.696296,56.993994,44.577377,9.0000,42.0000,16.0000,24.0000,11.0000
1,2016-04-01,1,1,60,5.9,43.8,0.3,0.0,5.6,1.1,0.0,1012.6,1015.3,9.2,34.0,1.2,0.0,-5.8,4.0,1011.7,1015.2,0.0,0.00,3.4,16.299999,34.0,0.5,0.0,16.400000,31.0,0.8,0.0,4,4,3,2,1,0,18.696296,56.993994,44.577377,8.2361,43.3841,14.3417,32.5948,10.6820
2,2016-04-01,1,1,120,5.5,45.5,0.3,0.0,5.1,0.0,0.0,1012.2,1014.9,9.2,33.0,1.1,0.0,-6.2,3.8,1011.3,1014.8,0.0,0.00,3.3,18.299999,29.0,0.8,0.0,18.700001,26.0,1.2,0.0,4,4,3,2,1,0,18.696296,56.993994,44.577377,7.7333,47.4309,14.0079,34.7600,10.2975
3,2016-04-01,1,1,180,5.2,52.1,0.2,0.0,4.7,0.3,0.0,1012.2,1014.9,7.5,40.0,0.4,0.0,-5.2,4.2,1011.4,1014.9,0.0,0.00,2.8,19.600000,24.0,0.7,0.0,20.200001,22.0,1.0,0.0,4,4,3,2,1,0,18.696296,56.993994,44.577377,7.0000,59.0000,14.0000,28.0000,10.0000
4,2016-04-01,1,1,240,4.7,47.1,0.0,0.0,3.7,1.1,0.0,1012.4,1015.1,4.8,56.0,0.6,0.0,-3.2,4.8,1011.6,1015.2,0.0,0.00,1.8,20.600000,25.0,2.2,0.0,21.500000,23.0,2.3,0.0,4,4,3,2,1,0,18.696296,56.993994,44.577377,6.4159,47.5708,13.7150,34.6242,10.2397
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210379,2020-03-31,2,3,1140,9.8,43.5,0.6,0.0,10.4,2.0,0.0,1012.9,1015.3,9.6,46.0,1.1,0.0,-1.4,5.5,1012.3,1015.4,0.0,0.08,10.3,8.500000,58.0,1.4,0.0,8.500000,58.0,1.4,0.0,4,4,3,4,1,4,17.982983,67.942943,29.674775,11.0199,43.8651,13.8034,35.3122,11.4007
210380,2020-03-31,2,3,1200,8.5,52.3,0.8,0.0,9.4,1.7,0.0,1013.4,1015.8,8.1,53.0,0.2,0.0,-0.9,5.7,1012.9,1016.0,0.0,0.00,8.9,8.300000,56.0,1.1,0.0,8.300000,56.0,1.1,0.0,4,4,3,4,1,4,17.982983,67.942943,29.674775,10.0869,47.0679,13.3667,37.1549,11.0804
210381,2020-03-31,2,3,1260,7.6,64.4,0.7,0.0,8.7,2.4,0.0,1013.9,1016.3,7.4,59.0,0.6,0.0,-0.1,6.1,1013.4,1016.5,0.0,0.00,8.0,8.400000,51.0,1.3,0.0,8.400000,51.0,1.3,0.0,4,4,3,4,1,4,17.982983,67.942943,29.674775,9.3965,60.0182,12.7654,42.6009,10.7654
210382,2020-03-31,2,3,1320,6.9,71.5,1.3,0.0,8.2,2.0,0.0,1013.9,1016.3,7.4,58.0,1.6,0.0,-0.3,6.0,1013.3,1016.4,0.0,0.00,7.8,9.400000,47.0,1.7,0.0,9.400000,47.0,1.7,0.0,4,4,3,4,1,4,17.982983,67.942943,29.674775,8.3909,56.9864,12.7285,42.6748,10.7028


In [ ]:
columns = ['date', 'time', 'dangjin-ta', 'dangjin-hm',
       'dangjin-ws', 'dangjin-rn', 'sinpyeong-ta', 'sinpyeong-ws',
       'sinpyeong-rn', 'sinpyeong-lp', 'sinpyeong-ap', 'seosan-ta',
       'seosan-hm', 'seosan-ws', 'seosan-rn', 'seosan-dew', 'seosan-sp',
       'seosan-lp', 'seosan-ap', 'seosan-sr', 'seosan-ss', 'seosan-ground',
       'songsan-ta', 'songsan-hm', 'songsan-ws', 'songsan-rn', 'songak-ta',
       'songak-hm', 'songak-ws', 'songak-rn']

In [ ]:
data = data[columns]

In [ ]:
FC_DATA = pd.merge(FC_DATA, data, how='left', on=['date', 'time'])

In [ ]:
FC_DATA = FC_DATA.dropna()

In [ ]:
FC_DATA.to_csv(os.path.join(DATA_CONV_PATH, 'ALL_FC_DATA_MEAN_ADD_WEATHER.csv'))

In [ ]:
MLA = [
       #ensemble.AdaBoostRegressor(),
       #ensemble.BaggingRegressor(),
       ensemble.ExtraTreesRegressor(),
       #ensemble.GradientBoostingRegressor(),
       #ensemble.RandomForestRegressor(),

       #gaussian_process.GaussianProcessClassifier(),

       #linear_model.LogisticRegressionCV(),
       #linear_model.PassiveAggressiveClassifier(),
       #linear_model.RidgeClassifierCV(),
       #linear_model.SGDClassifier(),
       #linear_model.Perceptron(),

       #naive_bayes.BernoulliNB(),
       #naive_bayes.GaussianNB(),

       #svm.SVC(probability=True),
       #svm.NuSVC(probability=True),
       #svm.LinearSVC(),

       #tree.DecisionTreeClassifier(),
       #tree.ExtraTreeClassifier(),

       #discriminant_analysis.LinearDiscriminantAnalysis(),
       #discriminant_analysis.QuadraticDiscriminantAnalysis(),

       #XGBClassifier()
       #lightgbm.LGBMClassifier()
]
MLA

[ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     max_samples=None, min_impurity_decrease=0.0,
                     min_impurity_split=None, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     n_estimators=100, n_jobs=None, oob_score=False,
                     random_state=None, verbose=0, warm_start=False)]

In [ ]:
data_columns = ['time', 'songsan_ta', 'songsan_hm', 'songsan_ws', 'songak_ta',
       'songak_hm', 'songak_ws', 'sinpyeong_ta', 'sinpyeong_hm',
       'sinpyeong_ws', 'dangjin1_ta', 'dangjin1_hm', 'dangjin1_ws',
       'dangjin2_ta', 'dangjin2_hm', 'dangjin2_ws', 'dangjin3_ta',
       'dangjin3_hm', 'dangjin3_ws', 'godae_ta', 'godae_hm', 'godae_ws',
       'emam_ta', 'emam_hm', 'emam_ws', 'susuk_ta', 'susuk_hm', 'susuk_ws',
       'suknam_ta', 'suknam_hm', 'suknam_ws']

In [ ]:
Targets = ['seosan-ta',
       'seosan-hm', 'seosan-ws', 'seosan-rn', 'seosan-dew', 'seosan-sp',
       'seosan-lp', 'seosan-sr', 'seosan-ss', 'seosan-ground',
       'songsan-ta', 'songsan-hm', 'songsan-ws']

In [ ]:
model = dict()

In [ ]:
import copy

Targets = Targets

data_train_bin = data_columns

cv_split = model_selection.ShuffleSplit(n_splits=10, test_size=.3, train_size=.6, random_state=0)

MLA_columns = ['MLA Name', 'MLA Parameters', 'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD', 'MLA Time']

MLA_compares = list()
MLA_predict = dict()
mla_index = 0
for Target in Targets:
  MLA_compare = pd.DataFrame(columns=MLA_columns)
  MLA_compares.append(MLA_compare)


  row_index = 0
  for alg in MLA:
    print(alg)
    data = copy.deepcopy(FC_DATA)
    MLA_name = alg.__class__.__name__
    MLA_compares[mla_index].loc[row_index, 'MLA Name'] = MLA_name
    MLA_compares[mla_index].loc[row_index, 'MLA Parameters'] = str(alg.get_params())

    cv_results = model_selection.cross_validate(alg, data[data_train_bin], data[Target], cv=cv_split)
    MLA_compares[mla_index].loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compares[mla_index].loc[row_index, 'MLA Test Accuracy Mean'] = cv_results['test_score'].mean()
    MLA_compares[mla_index].loc[row_index, 'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std()*3

    alg.fit(data[data_train_bin], data[Target])
    MLA_predict[(MLA_name, Target)] = alg.predict(data[data_train_bin])
    model[Target] = copy.deepcopy(alg)
    row_index += 1
  MLA_compares[mla_index].sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)
  mla_index += 1 

for idx in range(len(Targets)):
  print(MLA_compares[idx])

ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=None, oob_score=False,
                    random_state=None, verbose=0, warm_start=False)
ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                    max_depth=None, max_features='auto', max_leaf_nodes=None,
                    max_samples=None, min_impurity_decrease=0.0,
                    min_impurity_split=None, min_samples_leaf=1,
                    min_samples_split=2, min_weight_fraction_leaf=0.0,
                    n_estimators=100, n_jobs=None, oob_score=False,
                    random_state=None, verbose=0, warm_start=False)
ExtraTreesRegressor(bootstrap=Fa

In [ ]:
print('seosan-ta')
MLA_compares[0]

seosan-ta


,MLA Name,MLA Parameters,MLA Test Accuracy Mean,MLA Test Accuracy 3*STD,MLA Time
0,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",0.999813,0.00015321,20.1569


In [ ]:
print('seosan-hm')
MLA_compares[1]

seosan-hm


,MLA Name,MLA Parameters,MLA Test Accuracy Mean,MLA Test Accuracy 3*STD,MLA Time
0,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",0.997442,0.00158433,20.2704


In [ ]:
print('seosan-ws')
MLA_compares[2]

seosan-ws


,MLA Name,MLA Parameters,MLA Test Accuracy Mean,MLA Test Accuracy 3*STD,MLA Time
0,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",0.995725,0.0039823,21.2671


In [ ]:
print('seosan-rn')
MLA_compares[3]

seosan-rn


,MLA Name,MLA Parameters,MLA Test Accuracy Mean,MLA Test Accuracy 3*STD,MLA Time
0,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",0.992894,0.0315749,11.7685


In [ ]:
print('seosan-dew')
MLA_compares[4]

seosan-dew


,MLA Name,MLA Parameters,MLA Test Accuracy Mean,MLA Test Accuracy 3*STD,MLA Time
0,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",0.99968,0.000201562,19.4935


In [ ]:
print('seosan-sp')
MLA_compares[5]

seosan-sp


,MLA Name,MLA Parameters,MLA Test Accuracy Mean,MLA Test Accuracy 3*STD,MLA Time
0,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",0.999722,0.000272936,18.8166


In [ ]:
print('seosan-lp')
MLA_compares[6]

seosan-lp


,MLA Name,MLA Parameters,MLA Test Accuracy Mean,MLA Test Accuracy 3*STD,MLA Time
0,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",0.997706,0.00163457,20.7939


In [ ]:
print('seosan-sr')
MLA_compares[7]

seosan-sr


,MLA Name,MLA Parameters,MLA Test Accuracy Mean,MLA Test Accuracy 3*STD,MLA Time
0,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",0.996674,0.00312596,11.2139


In [ ]:
print('seosan-ss')
MLA_compares[8]

seosan-ss


,MLA Name,MLA Parameters,MLA Test Accuracy Mean,MLA Test Accuracy 3*STD,MLA Time
0,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",0.998563,0.00223648,13.6925


In [ ]:
print('seosan-ground')
MLA_compares[9]

seosan-ground


,MLA Name,MLA Parameters,MLA Test Accuracy Mean,MLA Test Accuracy 3*STD,MLA Time
0,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",0.999525,0.000698385,20.255


In [ ]:
print('songsan-ta')
MLA_compares[10]

songsan-ta


,MLA Name,MLA Parameters,MLA Test Accuracy Mean,MLA Test Accuracy 3*STD,MLA Time
0,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",0.999476,0.000443271,19.0688


In [ ]:
print('songsan-hm')
MLA_compares[11]

songsan-hm


,MLA Name,MLA Parameters,MLA Test Accuracy Mean,MLA Test Accuracy 3*STD,MLA Time
0,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",0.996054,0.00207423,19.6295


In [ ]:
print('songsan-ws')
MLA_compares[12]

songsan-ws


,MLA Name,MLA Parameters,MLA Test Accuracy Mean,MLA Test Accuracy 3*STD,MLA Time
0,ExtraTreesRegressor,"{'bootstrap': False, 'ccp_alpha': 0.0, 'criter...",0.992291,0.00605208,21.2499


In [ ]:
joblib.dump(model, os.path.join(MODEL_PATH, "model_fc_all.pkl"))

['/content/gdrive/My Drive/weather_bigcontest/4_code/model/model_fc_all.pkl']